In [1]:
import pandas as pd
import requests
import json

In [6]:
def generate_neutral_comment(toxic_comment, api_token):
    """
    Генерирует нейтральный вариант токсичного комментария используя GPT
    """
    url = "https://caila.io/api/mlpgate/account/just-ai/model/openai-proxy/predict"

    headers = {
        "Authorization": api_token,
        "Content-Type": "application/json",
        "MLP-API-KEY": api_token
    }

    # Модифицированный промпт
    payload = {
        "chat": {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "system",
                    "content": """You are an assistant that helps to rewrite potentially problematic comments in Russian language in a more constructive way.
                    Your task is to:
                    1. Maintain the main factual content
                    2. Remove emotional and toxic elements
                    3. Use neutral language
                    4. Keep the response concise
                    5. Avoid controversial topics
                    6. Write neutral variants in Russian only
                    If the content is too sensitive or inappropriate, return a general neutral statement."""
                },
                {
                    "role": "user",
                    "content": f"Please rewrite this comment in a neutral way: {toxic_comment}"
                }
            ]
        }
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()

        result = response.json()

        # Добавляем отладочную информацию
        print(f"API Response: {result}")

        if 'chat' in result and 'choices' in result['chat']:
            try:
                neutral_comment = result['chat']['choices'][0]['message']['content']
                return neutral_comment
            except KeyError as e:
                print(f"KeyError in response structure: {e}")
                print(f"Response structure: {result}")
                return "Could not process this comment"
        else:
            print("Unexpected response structure:", result)
            return "Could not process this comment"

    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
        return "Could not process this comment"
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "Could not process this comment"

In [7]:
def main():
    try:
        # Читаем исходный файл
        df = pd.read_csv('labeled.csv')

        # Фильтруем только токсичные комментарии (метка 1.0)
        toxic_comments = df[df.iloc[:, 1] == 1.0].iloc[:, 0].tolist()

        # API token
        api_token = input("Введите ваш API token: ")

        # Создаем список для хранения пар комментариев
        comment_pairs = []

        # Обрабатываем каждый токсичный комментарий
        for i, toxic_comment in enumerate(toxic_comments):
            print(f"\nProcessing comment {i+1}/{len(toxic_comments)}")
            print(f"Original comment: {toxic_comment[:100]}...")

            neutral_comment = generate_neutral_comment(toxic_comment, api_token)

            if neutral_comment:
                comment_pairs.append({
                    'toxic_comment': toxic_comment,
                    'neutral_comment': neutral_comment
                })
                print(f"Processed comment: {neutral_comment[:100]}...")

            # Сохраняем промежуточные результаты каждые 10 комментариев
            if (i + 1) % 10 == 0:
                temp_df = pd.DataFrame(comment_pairs)
                temp_df.to_csv(f'comment_pairs_temp_{i+1}.csv', index=False, encoding='utf-8-sig')

        # Создаем финальный DataFrame с парами комментариев
        result_df = pd.DataFrame(comment_pairs)

        # Сохраняем результаты в CSV
        result_df.to_csv('comment_pairs_final.csv', index=False, encoding='utf-8-sig')
        print("\nРезультаты сохранены в файл: comment_pairs_final.csv")

    except Exception as e:
        print(f"Error in main: {e}")

if __name__ == "__main__":
    main()

Выходные данные были обрезаны до нескольких последних строк (5000).
Original comment: Я видел, как жируху трахают в влажную подмышку.
...
API Response: {'chat': {'id': 'chatcmpl-AgVBiFpjiyzH7RCxInbnAUpyEgUev', 'object': 'chat.completion', 'created': 1734693026, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Я наблюдал необычное поведение людей.', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 136, 'completion_tokens': 11, 'total_tokens': 147, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'system_fingerprint': 'fp_0aa8d3e20b'}, 'spentMicroCents': 26999}
Processed comment: Я наблюдал необычное поведение людей....

Processing comment 3944/4826
Original comment: да эти татары тот еще народец, гнилой весьма и весьма...
...
API Respons